In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [15]:
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
# image = load_image("example_data/corgi.png")
# image = load_image("example_data/cube.png")
image = load_image("../../../content/cube_tall.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [4]:

# load latents from file
latents_cube = torch.load('../../latents/cube_latents.pt')
latents_cube_tall = torch.load('../../latents/cube_tall_latents.pt')

In [6]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
# size = 64 # this is the size of the renders; higher values take longer to render.
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))
    # save image to disk
    images[0].save('../../../content/cube_{}.gif'.format(i), save_all=True, append_images=images[1:], duration=100, loop=0)

NameError: name 'latents' is not defined

In [7]:
# function for interpolating among two latent vectors
def interpolate_latents(lat_A, lat_B, intp_steps):
    latents = []
    for i in range(intp_steps):
        latents.append(lat_A + (lat_B - lat_A) * i / intp_steps)
    return latents


In [16]:
# function for extrapolating from a latent vector
def extrapolate_latents(lat_A, lat_B, extp_steps):
    latents = []
    for i in range(extp_steps):
        latents.append(lat_B + (lat_B - lat_A) * i / extp_steps)
    return latents

In [ ]:
# function for extracting transformation from two latent vectors
def extract_transformation(lat_A, lat_B):
    return lat_B - lat_A

In [17]:
# function for rendering interpolated or extrapolated latents
def render_latents(latents, size, name):
    for i, latent in enumerate(latents, size):
        images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
        # display(gif_widget(images))
        # save image to disk
        images[0].save('../../../content/{}_{}.gif'.format(name, i), save_all=True, append_images=images[1:], duration=100, loop=0)

In [18]:
# interpolate between two latents
intp_latents = interpolate_latents(latents_cube[0], latents_cube_tall[0], 5)
extp_latents = extrapolate_latents(latents_cube[0], latents_cube_tall[0], 5)

In [19]:
render_latents(intp_latents, size, 'cube_intp')

In [ ]:
# save latents to file
save_latents = False
if save_latents:
    torch.save(latents, '../../latents/cube_tall_latents.pt')